# 인스타그램 로그인
<hr>
아래에 정의된 함수들을 한 번씩만 실시하고, 이 함수를 사용한 코드들을 모아둔 코드에서 크롤링을 실시하기
위의 코드들은 함수를 미리 정의하여, 크롤링 사용에 용이하도록 만든 코드들임

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re

# 크롬 브라우저 열기
driver = webdriver.Chrome('C:/Temp/chromedriver')

driver.get("https://www.instagram.com/accounts/login/")
time.sleep(2)

email = '본인 ID'
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = '본인 PWD' 
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()

In [ ]:
# 인스타그램 해시태그 검색

def insta_searching(word):
    url = "https://www.instagram.com/explore/tags/" + word +"/"
    return url
import time

word = "산수갑산"
url = insta_searching(word)
driver.get(url)

In [ ]:
# 총 게시물 개수

postnumber = driver.find_elements_by_css_selector("span > span.g47SY")[0].text.replace(',', '')
postnumber = int(postnumber)

In [ ]:
# 첫번째 게시물 열기

def select_first(driver):
    first = driver.find_element_by_css_selector("div._9AhH0")
    first.click()
    time.sleep(2)
    
select_first(driver)

In [ ]:
# 게시물 내용 크롤링

import re
from bs4 import BeautifulSoup

def get_content(driver):
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    # 본문 내용 
    try:
        content = soup.select('div.C4VMK > span')[0].text
    except:
        content = ' '
    # 해시태그 
    try:
        tags = re.findall(r'#[^\s#,\\]+', content)  
    except:
        tags = ' '
    # 작성일자 
    try:
        date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
    except:
        date = ''
    # 좋아요 
    try:
        like = soup.select('div.Nm9Fw > button')[0].text[4:-1]   
    except:
        like = 0
    # 위치
    try: 
        place = soup.select('div.M30cS')[0].text
    except:
        place = ''
   
    data = [content, date, like, place, tags]
    time.sleep(1)
    return data
    

get_content(driver)

In [ ]:
# 다음 게시물로 이동

def move_next(driver):

    right = driver.find_element_by_css_selector ('a.coreSpriteRightPaginationArrow')
    right.click()

move_next(driver)

# 인스타그램 게시물 크롤링 하기    <img src='data/instagram.PNG' width=120 height=120 align="right">         
### : 본격적인 크롤링은 아래의 코드를 사용하기
<hr>
위에 코드들을 한 번 씩 실행 후, 아래의 코드에 word 내용만 변경하여 실행하면 반복 실행 

In [ ]:
import pandas as pd

df = pd.read_excel('output/음식점 모음.xlsx').fillna("missing")

for i in range(30) :
    for j in range(20) :
        if df.iloc[i, j] != "missing":
            search = df.iloc[i, j].replace(' ', '')

            url = insta_searching(search)
            driver.get(url)
            time.sleep(8)

            postnumber = driver.find_elements_by_css_selector("span > span.g47SY")[0].text.replace(',', '')
            postnumber = int(postnumber)

            select_first(driver)
            instagram = []
            data = []

            for k in range(101) :
                html = driver.page_source
                soup = BeautifulSoup(html, 'lxml')
                # 본문 내용 
                try:
                    content = soup.select('div.C4VMK > span')[0].text
                except:
                    content = ' '
                # 해시태그 
                try:
                    tags = re.findall(r'#[^\s#,\\]+', content)  
                except:
                    tags = ' '
                # 작성일자 
                try:
                    date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
                except:
                    date = ''
                # 좋아요 
                try:
                    like = soup.select('div.Nm9Fw > button')[0].text[4:-1]   
                except:
                    like = 0
                # 위치
                try: 
                    place = soup.select('div.M30cS')[0].text
                except:
                    place = ''

                data = [content, date, like, place, tags]
                instagram = (instagram,data,'\n')
                time.sleep(0.5)
                move_next(driver)
                time.sleep(2)
           
            data_df = pd.DataFrame(instagram)
            data_df.columns = ['content','hashtag','date','like','place']
            data_df.to_excel('./data/instagram/'+ search +'.csv', encoding="utf-8")

# 데이터 전처리

In [ ]:
# 데이터 한 데 모으기
import os
import pandas as pd
from konlpy.tag import Okt  # kkma 사용하면, 개수가 안 맞음 

okt = Okt()     

total = pd.DataFrame()
temp = pd.DataFrame()
count_theName = pd.DataFrame()

path_dir = './data/instagram/rowdata'
file_list = os.listdir(path_dir)

# 데이터들이 들어있는 파일 열기
for l in file_list :
    tflist = pd.DataFrame()
    splited_dir = l.split('_')
    file = "./data/instagram/rowdata/" + l 
    temp_items = pd.read_csv(file, engine='python') # engine을 주지 않으면, encoding에러가 남

    for k in range(len(temp_items.index)) : 
        if splited_dir[1].replace(".csv", "") in (okt.nouns(temp_items.loc[k, "content"].replace("#","") + temp_items.loc[k, "hashtag"].replace("#",""))) :
            tf = pd.DataFrame(temp_items.loc[k]).transpose()
            tflist = tflist.append(tf, ignore_index=True)
            if tflist is not None :
                date = tflist.sort_values(by='date', ascending=True).head(1)
                theDay = str(date.iloc[0, 3])                
            else :
                theDay = "2099-12-31"
        oldestdate = temp_items.sort_values(by='date', ascending=True).head(1)
        oldestDate = str(oldestdate.iloc[0, 3])
        updateddate = temp_items.sort_values(by='date', ascending=False).head(1)
        updatedDate = str(updateddate.iloc[0, 3])

    tflist.to_csv('./data/instagram/referenceONLY/'+ splited_dir[0] + '_' + splited_dir[1], encoding="utf-8")    

    mediaName = splited_dir[1].replace(".csv", "")
    restaurant = splited_dir[0]
    data = pd.DataFrame([mediaName, restaurant, oldestDate, theDay, updatedDate]).transpose()         
    total = total.append(data, ignore_index = True)
    temp = temp.append(temp_items, ignore_index = True) 
                         
temp.to_csv('./data/instagram/sumData.csv', encoding="utf-8")
total.to_csv('./data/instagram/instagramtotal.csv', encoding="utf-8")

In [ ]:
# 인스타그램 동적 크롤링
import pandas as pd

df = pd.read_csv("./data/instagram/instagramtotal.csv")

postsNumber = []
#식당 별 인스타그램 게시물 개수 세기
for i in df["restaurant"] :
    url = "https://www.instagram.com/explore/tags/" + i +"/"
    driver.get(url)
    time.sleep(8)

    postnumber = driver.find_elements_by_css_selector("span > span.g47SY")[0].text.replace(',', '')
    postnumber = int(postnumber)
    postsNumber.append(postnumber)

print(postsNumber)
print(len(postsNumber))

df.to_csv("./data/instagram/instagramtotal.csv", index=False)

In [ ]:
import pandas as pd
import datetime

# 날짜 데이터 계산 및 변수 생성
df = pd.read_csv('./data/instagram/instagramtotal.csv')
df = df.drop([df.columns[0]], axis = 1)
df.columns = ['mediaName', 'restaurant', 'oldestDate', 'theDay', 'updatedDate']
display(df)

df.theDay = pd.to_datetime(df["theDay"])
df.oldestDate = pd.to_datetime(df["oldestDate"])
df.updatedDate = pd.to_datetime(df["updatedDate"])
df['before_theDay'] = (df.theDay - df.oldestDate).dt.days
df['after_theDay'] = (df.updatedDate - df.theDay).dt.days

display(df)

In [ ]:
# 식당 별 연예인, 방송 언급 횟수
import os
path_dir = './data/instagram/referenceONLY'
file_list = os.listdir(path_dir)

countName = []
for l in file_list :    
    splited_dir = l.split('_')
    file = "./data/instagram/referenceONLY/" + l 
    try :
        temp_items = pd.read_csv(file, engine='python')
        count = len(temp_items)
        countName.append(count)
    except :
        countName.append(0)
df["countName"] = countName
display(df)

In [ ]:
import os
import pandas as pd
from konlpy.tag import Okt  # kkma 사용하면, 개수가 안 맞음 

okt = Okt()     
reference_before = []
reference_after = []
total = pd.DataFrame()
temp = pd.DataFrame()
count_theName = pd.DataFrame()

path_dir = './data/instagram/rowdata'
file_list = os.listdir(path_dir)

# 데이터들이 들어있는 파일 열기
# 기준일 전 후로, 연예인 또는 방송 이름이 들어가 있는 열을 찾아서 개수 세기
for l in file_list :
    tflist = pd.DataFrame()
    splited_dir = l.split('_')
    file = "./data/instagram/rowdata/" + l 
    temp_items = pd.read_csv(file, engine='python') # engine을 주지 않으면, encoding에러가 남

    for k in range(len(temp_items.index)) : 
        if splited_dir[1].replace(".csv", "") in (okt.nouns(temp_items.loc[k, "content"].replace("#","") + temp_items.loc[k, "hashtag"].replace("#",""))) :
            tf = pd.DataFrame(temp_items.loc[k]).transpose()
            tflist = tflist.append(tf, ignore_index=True)
            date = tflist.sort_values(by='date', ascending=True).head(1)
            theDay = str(date.iloc[0, 3])                
    before = len(temp_items.loc[temp_items['date'] < theDay])
    reference_before.append(before)
    after = len(temp_items.loc[temp_items['date'] >= theDay])
    reference_after.append(after)

df["reference_before"] = reference_before
df["reference_after"] = reference_after
display(df)
df.to_csv("./data/instagram/instagramtotal.csv", index=False)

In [2]:
# 방송 종류 별, 연예인별로 묶기
import pandas as pd

df = pd.read_csv('./data/instagram/instagramtotal.csv')
연예인 = ["이영자", "백종원", "성시경", "신동엽", "최자", "비(정지훈)", "문세윤", "김준현"]
유튜버 = ["더들리", "츄릅켠"]
방송프로그램 = ["수요미식회", "생생정보통", "전참시", "생활의 달인", "6시 내 고향", "골목식당", "생방송 오늘 저녁", "모닝 와이드", "맛있는 녀석들", "생방송 투데이"]
groups = [연예인, 유튜버, 방송프로그램]
groups_names = ["연예인", "유튜버", "방송프로그램"]
for i, groups in enumerate(groups):
    for media in groups:
        df.loc[df["mediaName"] == media, "group"] = groups_names[i]
display(df)

df.to_csv("./data/instagram/instagramtotal.csv", index=False)